### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.parse
from selenium import webdriver
import time

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Browser Automation

In [2]:
#Open the website
driver = webdriver.Edge(executable_path= r'C:\Users\kana\Documents\Portofolio\Python\Data Scraping\Web Driver\edgedriver_win64\msedgedriver.exe')
driver.get('https://www.onthemarket.com/')
time.sleep(1)



C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [3]:
#Accept the Cookies
cookies = driver.find_element_by_xpath("//span/button[@class='otm-Button py-2 leading-normal h-auto shadow-none font-heading font-semibold text-center justify-center inline-flex items-center px-8 rounded-md border-2 disabled:bg-stone disabled:text-dove disabled:border-transparent transition duration-200 ease-in-out border-brand-contrast bg-brand-contrast text-white hover:opacity-80  min-h-[52px] text-md']")
cookies.click()

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [4]:
#Search property in London
search = driver.find_element_by_xpath('//input[@type="text"]')
search.click()
search.send_keys('London')

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [5]:
#Submit the search key
submit = driver.find_element_by_id('search-location-sale-btn')
submit.click()

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [14]:
#Get the url
current_url = driver.current_url
current_url

'https://www.onthemarket.com/for-sale/property/london/?view=grid'

### Scrape the Data 

In [15]:
#store the website params
headers = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'})

In [16]:
#Get request
response = requests.get(current_url, headers=headers)

In [17]:
#status code
response

<Response [200]>

In [18]:
#soup object
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Houses for sale in London | Property &amp; Houses to Buy | OnTheMarket
  </title>
  <meta content="user-scalable=yes width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/>
  <meta content="IE=EDGE" http-equiv="X-UA-Compatible">
   <meta content="yes" name="apple-mobile-web-app-capable"/>
   <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
   <meta content="#001229" name="theme-color"/>
   <meta content="noodp" name="robots">
    <script>
     dataLayer = [{"g-location-type":"country","g-location-id":"1010000005","property-prices":["265,000","290,000","325,000","360,000","370,000","475,000","475,000","480,000","499,950","500,000","500,000","575,000","600,000","615,000","625,000","625,000","640,000","650,000","675,000","700,000","825,000","1,100,000","8,500,000"],"no-of-properties":1000,"location-name":"London"

In [19]:
results = soup.find_all('li', {'class' : 'otm-PropertyCard isOwu'})

In [20]:
len(results)

22

In [21]:
address = []
beds = []
baths = []
prices = []

In [22]:
for result in results:
    address.append(result.find('span', {'class':'address'}).get_text())
    beds.append(result.find('div', {'class' : 'otm-BedBathCount flex items-center'}).find_all('div')[0].get_text())
    baths.append(result.find('div', {'class' : 'otm-BedBathCount flex items-center'}).find_all('div')[1].get_text())
    prices.append(result.find(attrs={"class": "mb-0 text-lg font-bold text-denim price"}).get_text())


In [26]:
button_next = driver.find_element_by_xpath('//a/button')
button_next.click()

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


### Output Pandas Dataframe

In [23]:
real_estate = pd.DataFrame(columns=['Address', 'Beds', 'Baths', 'Price'])

for i in range (len(address)):
    real_estate=real_estate.append({'Address':address[i], 'Beds':beds[i], 'Baths':baths[i], 'Price':prices[i]}, ignore_index=True)

In [24]:
real_estate

,Address,Beds,Baths,Price
0,"Ballards Way, Croydon, Surrey",4,-,"£600,000"
1,"Drayton Gardens, London",5,4,"£8,500,000"
2,"West Gardens, Colliers Wood",2,1,"£475,000"
3,"Monument Gardens, London SE13",3,1,"£370,000"
4,"Durham Road, London, SW20",2,-,"£625,000"
5,"Marion Road, Thornton Heath CR7",2,1,"£290,000"
6,"Rose Walk, West Wickham",4,2,"£650,000"
7,"Glasford Street, Tooting",2,1,"£499,950"
8,"The Avenue, Beckenham",2,1,"£700,000"
9,"Surrey Quays Road, Canada Water",1,1,"£475,000"


### Data Cleaning

In [134]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Address  22 non-null     object
 1   Beds     22 non-null     object
 2   Baths    22 non-null     object
 3   Price    22 non-null     object
dtypes: object(4)
memory usage: 832.0+ bytes


In [135]:
real_estate['Price'] = real_estate['Price'].apply(lambda x: x.strip('£').replace(",",""))
real_estate['Baths'] = real_estate['Baths'].apply(lambda x: x.replace("-", "0"))

In [136]:
#Convert the data
real_estate.Beds=real_estate.Beds.astype(int)
real_estate.Baths=real_estate.Baths.astype(int)
real_estate.Price=real_estate.Price.astype(int)

In [137]:
real_estate

,Address,Beds,Baths,Price
0,"Durham Road, London, SW20",2,0,625000
1,"Marion Road, Thornton Heath CR7",2,1,290000
2,"Rose Walk, West Wickham",4,2,650000
3,"Glasford Street, Tooting",2,1,499950
4,"Harrow Fields Gardens, Harrow on the Hill, HA1",4,2,775000
5,"The Avenue, Beckenham",2,1,700000
6,"Surrey Quays Road, Canada Water",1,1,475000
7,"Lyndhurst Grove, Peckham Rye",2,1,575000
8,"Lizban Street, Blackheath",2,1,615000
9,"Worsopp Drive, Clapham",1,1,325000


### Store in Excel

# Overall Code

In [39]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.parse
from selenium import webdriver
import time

#Open the website
driver = webdriver.Edge(executable_path= r'C:\Users\kana\Documents\Portofolio\Python\Data Scraping\Web Driver\edgedriver_win64\msedgedriver.exe')
driver.get('https://www.onthemarket.com/')
time.sleep(1)

#Accept the Cookies
cookies = driver.find_element_by_xpath("//span/button[@class='otm-Button py-2 leading-normal h-auto shadow-none font-heading font-semibold text-center justify-center inline-flex items-center px-8 rounded-md border-2 disabled:bg-stone disabled:text-dove disabled:border-transparent transition duration-200 ease-in-out border-brand-contrast bg-brand-contrast text-white hover:opacity-80  min-h-[52px] text-md']")
cookies.click()
time.sleep(1)

#Search property in London
search = driver.find_element_by_xpath('//input[@type="text"]')
search.click()
search.send_keys('London')
time.sleep(1)

#Submit the search key
submit = driver.find_element_by_id('search-location-sale-btn')
submit.click()
time.sleep(1)

address = []
beds = []
baths = []
prices = []


for i in range (1,6):
    
    #Get the url
    current_url = driver.current_url

    #store the website params
    headers = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'})

    #Get request
    response = requests.get(current_url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')
    if i == 1:  
        results = soup.find_all('li', {'class' : 'otm-PropertyCard isOwu'})
    else:
        results = soup.find_all('li', {'class' : 'otm-PropertyCard'})

    for result in results:
        address.append(result.find('span', {'class':'address'}).get_text())
        beds.append(result.find('div', {'class' : 'otm-BedBathCount flex items-center'}).find_all('div')[0].get_text())
        baths.append(result.find('div', {'class' : 'otm-BedBathCount flex items-center'}).find_all('div')[1].get_text())
        prices.append(result.find(attrs={"class": "mb-0 text-lg font-bold text-denim price"}).get_text())

    button_next = driver.find_element_by_xpath("//a[@title='Next page']/button")
    button_next.click()
    time.sleep(3)

C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.
C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app
C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\kana\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_elemen

In [77]:
real_estate = pd.DataFrame(columns=['Address', 'Beds', 'Baths', 'Price'])

for i in range (len(address)):
    real_estate=real_estate.append({'Address':address[i], 'Beds':beds[i], 'Baths':baths[i], 'Price':prices[i]}, ignore_index=True)
    
real_estate['Price'] = real_estate['Price'].apply(lambda x: x.strip('£').replace(",",""))
real_estate['Baths'] = real_estate['Baths'].apply(lambda x: x.replace("-", "0"))
real_estate['Beds'] = real_estate['Beds'].apply(lambda x: x.replace("-", "0"))




In [78]:
real_estate['Price'].replace("", 0, inplace=True)
real_estate['Beds'].replace("Studio", 1, inplace=True)

In [79]:
#Convert the data
real_estate.Beds=real_estate.Beds.astype(int)
real_estate.Baths=real_estate.Baths.astype(int)
real_estate.Price=real_estate.Price.astype(int)

In [80]:
real_estate

,Address,Beds,Baths,Price
0,"Ballards Way, Croydon, Surrey",4,0,600000
1,"Drayton Gardens, London",5,4,8500000
2,"West Gardens, Colliers Wood",2,1,475000
3,"Monument Gardens, London SE13",3,1,370000
4,"Durham Road, London, SW20",2,0,625000
...,...,...,...,...
109,"Battersea Roof Gardens, Battersea Power Statio...",1,1,925000
110,"Battersea Roof Gardens, Battersea Power Statio...",1,1,999000
111,"Prospect Place, Battersea Power Station, London",1,1,1048000
112,"Battersea Roof Gardens, Battersea Power Statio...",2,2,1750000


In [ ]:
del real_estate